# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [6]:
from pyspark.sql import SparkSession


# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
spark = SparkSession.builder.appName('quiz').getOrCreate()
# 3) read in the data set located at the path "data/sparkify_log_small.json"
data = spark.read.json("data/sparkify_log_small.json")
# 4) write code to answer the quiz questions 
data.head()

Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')

In [34]:
from pyspark.sql.functions import udf

# Question 1

Which page did user id "" (empty string) NOT visit?

In [13]:
# TODO: write your code to answer question 1
data.select('page').where(data.userId =='').dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [20]:
# TODO: use this space to explore the behavior of the user with an empty string
data.select(['auth','firstname','gender','level']).where(data.userId=='').collect()

[Row(auth='Logged Out', firstname=None, gender=None, level='free'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='free'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='free'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=None, level='paid'),
 Row(auth='Logged Out', firstname=None, gender=N

# Question 3

How many female users do we have in the data set?

In [23]:
# TODO: write your code to answer question 
data.select(['userId','gender']).dropDuplicates().groupBy('gender').count().collect()

[Row(gender='F', count=462),
 Row(gender=None, count=1),
 Row(gender='M', count=501)]

# Question 4

How many songs were played from the most played artist?

In [24]:
data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [37]:
# TODO: write your code to answer question 4
from pyspark.sql.types import IntegerType
song_count = udf(lambda song:1,IntegerType())
data_new = data.withColumn('song_count',song_count('song'))

In [76]:
from pyspark.sql.functions import desc
data.select(['artist','song']).groupBy('artist','song').agg({'song':'count'}) \
.withColumnRenamed('count(song)','count') \
.sort(desc('count')).show()

+--------------------+--------------------+-----+
|              artist|                song|count|
+--------------------+--------------------+-----+
|       Dwight Yoakam|      You're The One|   45|
|            BjÃÂ¶rk|                Undo|   44|
|Barry Tuckwell/Ac...|Horn Concerto No....|   28|
|         OneRepublic|             Secrets|   25|
|Florence + The Ma...|Dog Days Are Over...|   23|
|       Kings Of Leon|             Revelry|   22|
|            Harmonia|       Sehr kosmisch|   22|
|            Coldplay|              Yellow|   21|
|       Kings Of Leon|        Use Somebody|   19|
|    Five Iron Frenzy|              Canada|   17|
|            Tub Ring|             Invalid|   16|
|           Sam Cooke|    Ain't Misbehavin|   16|
|       Justin Bieber|    Somebody To Love|   13|
|        Travie McCoy|Billionaire [feat...|   13|
|  Charttraxx Karaoke|           Fireflies|   13|
| Angels and Airwaves|            The Gift|   12|
|            Coldplay|              Clocks|   12|


In [74]:
data.select(['artist']).groupBy('artist').agg({'artist':'count'}) \
.sort(desc('count(artist)')) \
.show(1)

+--------+-------------+
|  artist|count(artist)|
+--------+-------------+
|Coldplay|           83|
+--------+-------------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [77]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [79]:
# TODO: write your code to answer question 5

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = data.filter((data.page == 'NextSong') | (data.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

